In [ ]:
from ada import Assembly, Beam, Material, Part
from ada.fem import Bc, FemSet, Step
from ada.fem.meshing import GmshOptions
from ada.fem.utils import get_beam_end_nodes
from ada.materials.metals import CarbonSteel

from ada.visualize.femviz import visualize_it

In [ ]:
bm = Beam("MyBeam", (0, 0.5, 0.5), (5, 0.5, 0.5), "IPE400", Material("S420", CarbonSteel("S420")))

In [ ]:
def make_fem(beam: Beam, geom_repr):
    part = bm.parent
    assembly = part.get_assembly()
    part.fem = bm.to_fem_obj(0.1, geom_repr, options=GmshOptions(Mesh_ElementOrder=1))
    fix_set = part.fem.add_set(FemSet("bc_nodes", get_beam_end_nodes(beam), FemSet.TYPES.NSET))
    assembly.fem.add_bc(Bc("Fixed", fix_set, [1, 2, 3, 4, 5, 6]))
    assembly.fem.add_step(Step("Eigen", Step.TYPES.EIGEN, eigenmodes=10))


a = Assembly("MyAssembly") / [Part("MyPart") / bm]
make_fem(bm, "line")
res = a.to_fem("Cantilever_CA_EIG_bm", "code_aster", overwrite=True, execute=True)

In [ ]:
visualize_it(res.results_file_path, temp_file="../_temp_bm.vtu")

In [ ]:
for mode in res.eigen_mode_data.modes:
    print(mode)

In [ ]:
a = Assembly("MyAssembly") / [Part("MyPart") / bm]
make_fem(bm, "shell")
res = a.to_fem("Cantilever_CA_EIG_sh", "code_aster", overwrite=True, execute=True)

In [ ]:
visualize_it(res.results_file_path, temp_file="../_temp_sh.vtu")

In [ ]:
for mode in res.eigen_mode_data.modes:
    print(mode)